In [2]:
!pip install ortools
!pip install dtale

     |████████████████████████████████| 32.7MB 104kB/s 
     |████████████████████████████████| 7.6MB 5.4MB/s 
     |████████████████████████████████| 112kB 45.7MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 71kB 7.6MB/s 
     |████████████████████████████████| 1.8MB 41.6MB/s 
     |████████████████████████████████| 645kB 42.2MB/s 
     |████████████████████████████████| 358kB 38.6MB/s 
     |████████████████████████████████| 1.2MB 44.1MB/s 
     |████████████████████████████████| 3.4MB 39.2MB/s 
     |████████████████████████████████| 194kB 47.0MB/s 
     |████████████████████████████████| 1.8MB 35.7MB/s 
  Created wheel for dash-bootstrap-components: filename=dash_bootstrap_components-0.10.3-cp36-none-any.whl size=179484 sha256=b25ae1ed556c1d4fb4081bb22845157e108879cdb5863aa5cffce140cbea685a
  Stored in dire

In [3]:
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model
from dtale import show
import dtale.app as dtale_app
dtale_app.USE_COLAB = True

In [9]:
# Clone the entire repo.
!git clone -l -s https://github.com/mswastik/optimization.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 35 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (35/35), done.
/content/cloned-repo/cloned-repo/cloned-repo
 CP.ipynb  'item master.xlsx'   orders.xlsx   README.md


In [13]:
df=pd.read_excel('orders.xlsx')

In [14]:
df = df[df['Line']=='Tmmthpkhti Limi']
del df['Line']
df['due'] = df['Dispatch Date'] - df['Dispatch Date'].min()
df['due'] = df['due'].dt.days
df.reset_index(inplace=True,drop=True)
df['Key']=df['SO'].astype('str')+df['FG Code']

In [ ]:
show(df,ignore_duplicate=True)

https://pi6pg94wws-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/3

In [15]:
df

,SO,Customer,SO Date,Dispatch Date,FG Code,Order Qty,Description,Plant,due,Key
0,17023287,mWI,2019-01-14,2019-04-01,gd29904300iX,486.0,grihhdimt glumridi Gil TPhT 43 G (1.5) m,1024,0,17023287gd29904300iX
1,17023287,mWI,2019-01-14,2019-04-01,gd29907800iX,216.0,grihhdimt glumridi Gil TPhT 78G- 2.75 mZ,1024,0,17023287gd29907800iX
2,17023287,mWI,2019-01-14,2019-04-01,gd39000800iX,360.0,grihhdimt kmtickvity gLd GiL TPhT 0.28mZ,1024,0,17023287gd39000800iX
3,17023287,mWI,2019-01-14,2019-04-01,gd55102400iX,180.0,didichmici Pridiud gld TPhT 24g-0.85 mZ,1024,0,17023287gd55102400iX
4,17023287,mWI,2019-01-14,2019-04-01,gd30805600iX,216.0,grihhdimt dimturi kdhihivi 56.7G-20Z-dk2,1025,0,17023287gd30805600iX
...,...,...,...,...,...,...,...,...,...,...
221,17023468,mWI,2019-04-24,2019-06-24,gd28001700iX,640.0,grihhdimt glumridi TPhT 17g (0.6) mZ,1024,84,17023468gd28001700iX
222,17023833,mWI,2019-04-23,2019-06-24,gd55001700iX,320.0,didichmici glumridi TPhT 17g-0.6 mZ,1024,84,17023833gd55001700iX
223,17023833,mWI,2019-04-23,2019-06-24,gd55102400iX,360.0,didichmici Pridiud gld TPhT 24g-0.85 mZ,1024,84,17023833gd55102400iX
224,17023833,mWI,2019-04-23,2019-06-24,gd30212200iX,720.0,grihhdimt himhitivi TPhT122G-4.3mz-TPh43,1024,84,17023833gd30212200iX


In [16]:
jobs =df['Key']
op =[1,2,3,4]
mc = [1,2,3,4,5,6]
sodate = df['SO Date']- df['Dispatch Date'].min()
sodate = sodate.dt.days
deldate = df['due']



In [17]:
sodate

0     -77
1     -77
2     -77
3     -77
4     -77
       ..
221    23
222    22
223    22
224    22
225    14
Name: SO Date, Length: 226, dtype: int64